# SQL Server Troubleshooting Guide

## Tampa Bay SQL Server User Group (TSUG)

Last Updated 10 March 2023

### Troubleshooting Process and Other Resources

[General troubleshooting process steps from Cisco](https://www.ciscopress.com/articles/article.asp?p=2999386&seqNum=5](https://www.ciscopress.com/articles/article.asp?p=2999386&seqNum=5)
- Step 1. Identify the symptoms of the problem.
- Step 2. Establish a theory of probable cause.
- Step 3. Investigate the theory to determine the cause.
- Step 4. Establish a plan of action to resolve the problem and implement the solution.
- Step 5. Verify full system functionality and, if applicable, implement preventive measures.
- Step 6. Document findings, actions, and outcomes.
  
#### Other Sources:

- [Visual Studio Code/Azure Data Studio add-in at this location with additional diagnostic Notebooks](https://github.com/EmanueleMeazzo/tsql.tech-Code-snippets/releases/tag/v1.0)
- [Microsoft Documentation on SQL Server Troubleshooting](https://learn.microsoft.com/en-us/troubleshoot/sql/welcome-sql-server)


### Connection Issues

_(Source- [https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/connect/resolve-connectivity-errors-overview](https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/connect/resolve-connectivity-errors-overview))_

The above article is the best place to start for connection issues. This section uses tools and processes defined in that documentation as of 10 March 2023

In [ ]:
/* 
1. Ping the server's name, then IP address 
ping localhost
ping 10.1.1.1
*/

/* 
2. Check using UDL Files: 
https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/connect/test-oledb-connectivity-use-udl-file 
*/

/*
3. Download the PortQry and PortQueryUI tools:
https://www.microsoft.com/en-us/download/details.aspx?id=17148
https://www.microsoft.com/en-us/download/details.aspx?id=24009 
Use: 
https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/connect/using-portqrytool-sqlserver
*/


### SQL Server Instance Information and Status

In [2]:
/* Find running services on the server 
Source: https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/system-dynamic-management-views?view=sql-server-ver16
*/

SELECT *
FROM sys.dm_server_services;
GO

(4 rows affected)

Total execution time: 00:00:00.037

servicename,startup_type,startup_type_desc,status,status_desc,process_id,last_startup_time,service_account,filename,is_clustered,cluster_nodename,instant_file_initialization_enabled
SQL Server (MSSQLSERVER),2,Automatic,4,Running,6408,2023-02-10 07:59:57.1430157 -05:00,NT Service\MSSQLSERVER,"""C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Binn\sqlservr.exe"" -sMSSQLSERVER",N,NULL,N
SQL Server Agent (MSSQLSERVER),3,Manual,4,Running,27584,NULL,NT Service\SQLSERVERAGENT,"""C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Binn\SQLAGENT.EXE"" -i MSSQLSERVER",N,NULL,N
SQL Full-text Filter Daemon Launcher (MSSQLSERVER),3,Manual,4,Running,8756,NULL,NT Service\MSSQLFDLauncher,"""C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Binn\fdlauncher.exe"" -s MSSQL15.MSSQLSERVER",N,NULL,N
SQL Server Launchpad (MSSQLSERVER),2,Automatic,4,Running,8856,NULL,NT Service\MSSQLLaunchpad,"""C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Binn\launchpad.exe"" -launcher RLauncher.dll -launcher Pythonlauncher.dll -launcher commonlauncher.dll -pipename sqlsatellitelaunch -timeout 600000 -logPath ""C:\Program Files\Microsoft SQL Ser",N,NULL,N


### SQL Server Databases Information and Status

In [15]:
/* Databases and their statuses
Source: https://learn.microsoft.com/en-us/sql/relational-databases/system-catalog-views/sys-databases-transact-sql?view=sql-server-ver16 
*/
SELECT name
, state_desc AS 'Status'
, user_access_desc
, compatibility_level
, create_date
, is_in_standby
, is_cleanly_shutdown
, recovery_model_desc
, is_encrypted


FROM sys.databases;
GO


(17 rows affected)

Total execution time: 00:00:00.050

name,Status,user_access_desc,compatibility_level,create_date,is_in_standby,is_cleanly_shutdown,recovery_model_desc,is_encrypted
master,ONLINE,MULTI_USER,150,2003-04-08 09:13:36.390,0,0,SIMPLE,0
tempdb,ONLINE,MULTI_USER,150,2023-02-10 08:00:02.147,0,0,SIMPLE,0
model,ONLINE,MULTI_USER,150,2003-04-08 09:13:36.390,0,0,FULL,0
msdb,ONLINE,MULTI_USER,150,2019-09-24 14:21:42.270,0,0,SIMPLE,0
DWDiagnostics,ONLINE,MULTI_USER,150,2021-05-05 09:22:16.723,0,0,SIMPLE,0
DWConfiguration,ONLINE,MULTI_USER,150,2021-05-05 09:22:18.433,0,1,FULL,0
DWQueue,ONLINE,MULTI_USER,150,2021-05-05 09:22:19.273,0,1,SIMPLE,0
ReportServer,ONLINE,MULTI_USER,150,2021-05-05 09:27:13.493,0,0,FULL,0
ReportServerTempDB,ONLINE,MULTI_USER,150,2021-05-05 09:27:13.863,0,0,SIMPLE,0
AdventureWorks,ONLINE,MULTI_USER,140,2021-08-20 09:54:14.527,0,0,SIMPLE,0


### Database Backups Status

In [16]:
/* Check the backups before you do anything else!
Source: https://dba.stackexchange.com/questions/150241/t-sql-query-for-date-of-last-full-backup-size-and-location
*/
WITH LastBackUp AS
(
SELECT  bs.database_name,
        bs.backup_size,
        bs.backup_start_date,
        bmf.physical_device_name,
        Position = ROW_NUMBER() OVER( PARTITION BY bs.database_name ORDER BY bs.backup_start_date DESC )
FROM  msdb.dbo.backupmediafamily bmf
JOIN msdb.dbo.backupmediaset bms ON bmf.media_set_id = bms.media_set_id
JOIN msdb.dbo.backupset bs ON bms.media_set_id = bs.media_set_id
WHERE   bs.[type] = 'D'
AND bs.is_copy_only = 0
)
SELECT 
        sd.name AS [Database],
        CAST(backup_size / 1048576 AS DECIMAL(10, 2) ) AS [BackupSizeMB],
        backup_start_date AS [Last Full DB Backup Date],
        physical_device_name AS [Backup File Location]
FROM sys.databases AS sd
LEFT JOIN LastBackUp AS lb
    ON sd.name = lb.database_name
    AND Position = 1
ORDER BY [Database];

(17 rows affected)

Total execution time: 00:00:00.032

Database,BackupSizeMB,Last Full DB Backup Date,Backup File Location
AdventureWorks,207.09,2021-09-01 10:45:13.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\AdventureWorks_backup_2021_09_01_104512_5926035.bak
AdventureWorksDW,97.09,2021-09-01 10:45:13.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\AdventureWorksDW_backup_2021_09_01_104512_5926035.bak
Analysis,290.09,2021-09-01 10:45:14.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\Analysis_backup_2021_09_01_104512_5926035.bak
DWConfiguration,5.09,2021-09-01 10:45:12.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\DWConfiguration_backup_2021_09_01_104512_5926035.bak
DWDiagnostics,11.09,2021-09-01 10:45:12.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\DWDiagnostics_backup_2021_09_01_104512_5711248.bak
DWQueue,3.90,2021-09-01 10:45:13.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\DWQueue_backup_2021_09_01_104512_5926035.bak
master,NULL,NULL,NULL
model,NULL,NULL,NULL
msdb,NULL,NULL,NULL
Northwind,6.90,2021-09-01 10:45:17.000,C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\Backup\Northwind_backup_2021_09_01_104512_5926035.bak


### SQL Server Logs

In [ ]:
/* Read Error Logs in SQL Server
Source: https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/startup-shutdown/manage-error-log/ 
0 means current log, use higher numbers to go backwards
*/
EXEC sp_readerrorlog 0, 1;
GO

### SQL Server Agent Logs

In [ ]:
/* Read Error Logs in SQL Server
Source: https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/startup-shutdown/manage-error-log/ 
0 means current log, use higher numbers to go backwards
*/
EXEC sp_readerrorlog 0, 2;
GO